In [ ]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local").getOrCreate()

from pyspark.sql.functions import explode, collect_list, size, col, row_number, sort_array, count, udf,when
from pyspark.sql.types import ArrayType, IntegerType
from pyspark.sql import Window
from pyspark.sql.types import *

cSchema = StructType([StructField("x", StringType()),
                      StructField("distance", IntegerType())])

distances_list=[[12, 0]]

distances = sparkSession.createDataFrame(distances_list,schema=cSchema) 

customSchema = StructType([StructField("user", StringType()),
                           StructField("follower", StringType())])
graphData = sparkSession.read.option("delimiter", "\t").schema(customSchema).csv("/data/twitter/twitter_sample.txt")

forward_edges = graphData.select(col("follower").alias("x1"), 
                                 col("user").alias("x2")).persist()
count_x = 0
while count_x == 0:
    candidates = distances.join(forward_edges, col("x")==col("x1"), "inner")\
                          .select(col("x2").alias("x"), (col("distance") + 1).alias("new_distance")).persist()
    new_distances = distances.join(candidates, "x", "outer")\
                             .select("x", when(col("distance").isNotNull(), col("distance"))\
                                            .otherwise(col("new_distance")).alias("distance")).persist()
    distances = new_distances
    check_x = new_distances.filter(col("x") == 34)
    count_x = check_x.count()
    
result = check_x.orderBy("distance").take(1)
print(result[0].distance)
#check_x.collect()